In [19]:
# Uncomment to upgrade packages
#!pip3 install pandas --upgrade --quiet
#!pip3 install numpy  --upgrade --quiet
#!pip3 install scipy --upgrade --quiet
#!pip3 install statsmodels  --upgrade --quiet
#!pip3 install seaborn  --upgrade --quiet
#!pip3 install matplotlib  --upgrade --quiet
#!pip3 install scikit-learn  --upgrade  --quiet
#!pip install scikit-optimize  --quiet
#!pip install -U --quiet yellowbrick

In [20]:
# Imports de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns      
import missingno as msno
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures

from apafib import load_medical_costs  
from sklearn.model_selection import train_test_split
from scipy import stats
from matplotlib.lines import Line2D

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Definiciones
RND = 16 

sns.set(style="whitegrid", font_scale=1.05)

# Funciones auxiliares
def format_pval(p):
    """Formatea p-value: decimal si p >= 1e-3, si no en notación mantisa*10^{exp} (LaTeX)."""
    if p == 0:
        return r"$0$"
    exp = int(np.floor(np.log10(p)))
    if exp >= -3:
        return f"{p:.4f}"                # p mostrado como decimal (4 decimales)
    else:
        mant = p / (10.0**exp)
        # devolver string LaTeX, por ejemplo: $1.94\times10^{-19}$
        return rf"${mant:.2f}\times10^{{{exp}}}$"
    

from time import time
from datetime import timedelta
init_time = time()


# quitar warnings:
import warnings
warnings.filterwarnings("ignore", message="findfont:.*")

# Práctica de APA

Nosotros haremos un modelo de ML para predecir el ganador de partidos de tenis. Utilizaremos el repo de Jeff Sackman entre 2011 y 2024 como dataset (estamos hablando de más de 39k filas de datos). Es un problema de clasificación (binaria).

In [21]:
matches = pd.read_csv('./data/atp_matches.csv')
matches.head()
matches.shape

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,month,day,month_name,tourney_points
0,2011-339,Brisbane,Hard,32,A,20110102,1,104417,1.0,NaN,...,4.0,5.0,5580.0,173.0,309.0,2011,1,2,January,250
1,2011-339,Brisbane,Hard,32,A,20110102,2,103582,NaN,NaN,...,5.0,58.0,835.0,75.0,643.0,2011,1,2,January,250
2,2011-339,Brisbane,Hard,32,A,20110102,3,105051,NaN,Q,...,8.0,196.0,263.0,204.0,243.0,2011,1,2,January,250
3,2011-339,Brisbane,Hard,32,A,20110102,4,104797,8.0,NaN,...,3.0,40.0,1031.0,43.0,975.0,2011,1,2,January,250
4,2011-339,Brisbane,Hard,32,A,20110102,5,103888,4.0,NaN,...,6.0,16.0,1991.0,83.0,600.0,2011,1,2,January,250


(39541, 54)

Sabemos que tendremos mucho trabajo con el preprocesado de los datos, incluso haciendo feature engineering para añadir posibles variables útiles.

Haremos entonces lo que nos dijiste en clase: entregar la precisión del modelo de ML (para asegurarnos que es mejor que la aleatoriedad: >50% de precisión).

Obviamente nos reservamos el derecho para poder añadir features que puedan ser interesantes, para ver a qué nivel de precisión somos capaces de llegar: 70%? 80%?

In [22]:
display(matches.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
tourney_id,39541,1894,2011-580,127,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tourney_name,39541,1078,Australian Open,1778,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surface,39488,4,Hard,23060,NaN,NaN,NaN,NaN,NaN,NaN,NaN
draw_size,39541.0,NaN,NaN,NaN,57.467489,42.144138,2.0,32.0,32.0,96.0,128.0
tourney_level,39541,6,A,21148,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tourney_date,39541.0,NaN,NaN,NaN,20174404.064313,41143.884302,20110102.0,20140319.0,20170731.0,20210726.0,20241218.0
match_num,39541.0,NaN,NaN,NaN,167.239802,151.260611,1.0,21.0,190.0,283.0,1701.0
winner_id,39541.0,NaN,NaN,NaN,117737.747477,29694.476989,100644.0,104607.0,105373.0,109739.0,212721.0
winner_seed,16884.0,NaN,NaN,NaN,7.543592,6.988957,1.0,3.0,5.0,9.0,33.0
winner_entry,5347,10,Q,2968,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vamos a empezar limpiando los datos.

- Se eliminan identificadores personales: winner_id, loser_id, winner_name, loser_name, winner_ioc, loser_ioc.

- Se eliminan day y month_name porque la fecha ya está en tourney_date.

- Se eliminan las variables de seed (winner_seed, loser_seed), ya que pueden derivarse del ranking ATP cercano al partido.

- Se elimina tourney_level.

In [23]:
matches.drop(columns=['tourney_id', 'winner_id', 'loser_id', 'day', 'month_name', 'loser_seed', 'winner_seed', 'tourney_level'], inplace=True)

### Control del Data Leakage

En el contexto del aprendizaje automático, el *data leakage* sucede cuando el modelo tiene acceso a datos que, en la vida real, no debería conocer al momento de hacer predicciones. Esto incluye información futura o datos que dependen directamente del resultado que estamos intentando predecir.

En nuestro caso, las fugas pueden presentarse de tres formas:

1. **Temporal** → si usamos partidos futuros para generar estadísticas de los jugadores actuales.  
   Solución: ordenar cronológicamente los partidos antes de calcular cualquier feature.

2. **Estructural** → si el modelo distingue sistemáticamente entre el jugador etiquetado como “winner” y “loser”.  
   Solución: crear una estructura simétrica `player1/player2` aleatorizada para evitar sesgos.

3. **De variables** → si incluimos estadísticas generadas después del partido (p. ej. `winner_aces`).  
   Solución: eliminar columnas con datos del resultado o dependientes de él.

Controlar el *data leakage* es esencial para garantizar que el modelo aprenda patrones reales y generalizables, y no una visión privilegiada del pasado.


In [24]:
import numpy as np

# Ordenar cronológicamente ANTES del shuffle (crítico para evitar data leakage temporal)
matches = matches.sort_values('tourney_date').reset_index(drop=True)
print("DataFrame ordenado por fecha.")

# Configurar semilla para reproducibilidad
np.random.seed(RND)

# Crear máscara de shuffle: True = mantener orden, False = invertir
shuffle_mask = np.random.rand(len(matches)) > 0.5

# Inicializar columnas player1 y player2
# Identificar todas las columnas que empiezan con winner_ o loser_
winner_cols = [col for col in matches.columns if col.startswith('winner_')]
loser_cols = [col for col in matches.columns if col.startswith('loser_')]

# Crear diccionario de mapeo
winner_to_player = {col: col.replace('winner_', 'player1_') for col in winner_cols}
loser_to_player = {col: col.replace('loser_', 'player2_') for col in loser_cols}

# Copiar y renombrar columnas
for winner_col, player1_col in winner_to_player.items():
    matches[player1_col] = matches[winner_col]
    
for loser_col, player2_col in loser_to_player.items():
    matches[player2_col] = matches[loser_col]

# Aplicar shuffle: intercambiar player1 y player2 según la máscara
print(f"Aplicando shuffle aleatorio a {(~shuffle_mask).sum()} de {len(matches)} partidos...")

for i in range(len(matches)):
    if not shuffle_mask[i]:  # Si shuffle_mask[i] es False, intercambiar
        # Intercambiar todas las columnas player1 y player2
        for winner_col in winner_cols:
            player1_col = winner_col.replace('winner_', 'player1_')
            player2_col = winner_col.replace('winner_', 'player2_')
            
            # Swap
            temp = matches.at[i, player1_col]
            matches.at[i, player1_col] = matches.at[i, player2_col]
            matches.at[i, player2_col] = temp

# Crear variable target: player1_wins
# Si shuffle_mask[i] es True, player1 es el winner original (ganó)
# Si shuffle_mask[i] es False, player1 es el loser original (perdió)
matches['player1_wins'] = shuffle_mask.astype(int)

print(f"\n✓ Shuffle completado")
print(f"  - player1_wins = 1: {matches['player1_wins'].sum()} partidos ({matches['player1_wins'].mean()*100:.1f}%)")
print(f"  - player1_wins = 0: {(1-matches['player1_wins']).sum()} partidos ({(1-matches['player1_wins'].mean())*100:.1f}%)")
print(f"\nPrimeras filas con nueva estructura:")
display(matches[['player1_name', 'player2_name', 'player1_wins']].head(10))

DataFrame ordenado por fecha.
Aplicando shuffle aleatorio a 19842 de 39541 partidos...

✓ Shuffle completado
  - player1_wins = 1: 19699 partidos (49.8%)
  - player1_wins = 0: 19842 partidos (50.2%)

Primeras filas con nueva estructura:


,player1_name,player2_name,player1_wins
0,Ryan Harrison,Robin Soderling,0
1,Michael Berrer,Dudi Sela,1
2,Matthew Ebden,John Millman,1
3,Thiemo De Bakker,Denis Istomin,0
4,Adrian Mannarino,Mardy Fish,0
5,Tobias Kamke,Radek Stepanek,0
6,Ricardas Berankis,Arnaud Clement,1
7,Bernard Tomic,Florian Mayer,0
8,Philipp Petzschner,Feliciano Lopez,0
9,Kevin Anderson,Peter Luczak,1


## Shuffle aleatorio: winner/loser → player1/player2

**¿Por qué es necesario?**

Si mantenemos la estructura `winner_*` vs `loser_*`, el modelo aprenderá simplemente que "el jugador etiquetado como winner siempre gana", lo cual es **data leakage implícito**.

**Solución:**
1. Para cada partido, shuffle aleatorio de quién es `player1` y quién es `player2`
2. Crear variable target `player1_wins` (1 si player1 ganó, 0 si perdió)
3. El modelo aprende de características reales, no de la estructura de los datos

## Feature Engineering

Variables que causarían leakage:
- Todas las columnas que empiezan por `winner_` o `loser_` (ya transformadas a `player1`/`player2`)
- `score`
- `minutes`

Estrategia:
- Trabajar siempre con la estructura `player1` vs `player2`.
- Crear nuevas variables a partir de estas fuentes, evitando exponer información del resultado.
- Eliminar las columnas de leakage tras generar las nuevas características.

### 1. H2H (Head-to-Head)

Calculamos el historial de enfrentamientos directos entre los dos jugadores. Para cada partido se obtiene cuántas veces un jugador ha ganado al otro anteriormente. Esta feature añade contexto más allá de las estadísticas individuales, pues por estilo de juego ciertos jugadores dominan a otros.

In [25]:
# Feature #1: H2H (Head-to-Head)
from collections import defaultdict

# --- Preparar la estructura para guardar los récords H2H ---
# La clave será una tupla ordenada con los nombres de los dos jugadores.
# El valor será un contador de victorias para cada uno en ese enfrentamiento.
h2h_records = defaultdict(lambda: defaultdict(int))

# Listas para guardar los resultados calculados
player1_h2h_list = []
player2_h2h_list = []

# --- Iterar sobre cada partido para calcular el H2H ---
print("Calculando H2H para cada partido...")
for idx, row in matches.iterrows():
    player1_name = row['player1_name']
    player2_name = row['player2_name']
    player1_won = row['player1_wins']
    
    # Crear una clave única y ordenada para el par de jugadores
    player_pair_key = tuple(sorted((player1_name, player2_name)))
    
    # Consultar el historial H2H *antes* de este partido
    player1_wins_before = h2h_records[player_pair_key][player1_name]
    player2_wins_before = h2h_records[player_pair_key][player2_name]
    
    # Guardar los valores históricos
    player1_h2h_list.append(player1_wins_before)
    player2_h2h_list.append(player2_wins_before)
    
    # Actualizar el registro con el resultado de *este* partido para el futuro
    if player1_won == 1:
        h2h_records[player_pair_key][player1_name] += 1
    else:
        h2h_records[player_pair_key][player2_name] += 1

# --- Añadir las nuevas columnas al DataFrame ---
matches['player1_h2h_wins'] = player1_h2h_list
matches['player2_h2h_wins'] = player2_h2h_list
print("✓ Nuevas columnas H2H añadidas al DataFrame.")

# --- Verificar el resultado ---
print("\nPrimeros 10 partidos:")
print(matches[['player1_name', 'player2_name', 'player1_h2h_wins', 'player2_h2h_wins', 'player1_wins']].head(10))
print("\nÚltimos 10 partidos:")
print(matches[['player1_name', 'player2_name', 'player1_h2h_wins', 'player2_h2h_wins', 'player1_wins']].tail(10))

Calculando H2H para cada partido...
✓ Nuevas columnas H2H añadidas al DataFrame.

Primeros 10 partidos:
         player1_name     player2_name  player1_h2h_wins  player2_h2h_wins  \
0       Ryan Harrison  Robin Soderling                 0                 0   
1      Michael Berrer        Dudi Sela                 0                 0   
2       Matthew Ebden     John Millman                 0                 0   
3    Thiemo De Bakker    Denis Istomin                 0                 0   
4    Adrian Mannarino       Mardy Fish                 0                 0   
5        Tobias Kamke   Radek Stepanek                 0                 0   
6   Ricardas Berankis   Arnaud Clement                 0                 0   
7       Bernard Tomic    Florian Mayer                 0                 0   
8  Philipp Petzschner  Feliciano Lopez                 0                 0   
9      Kevin Anderson     Peter Luczak                 0                 0   

   player1_wins  
0             0  
1

### 2. Victorias y derrotas totales y anuales
Calculamos el rendimiento tanto histórico como del año en curso de cada jugador. Esta información permite al modelo evaluar la forma general del jugador y su consistencia a lo largo del tiempo. Por otro lado, las estadísticas del año actual (Year-to-Date) ayudan a capturar tendencias y estado de forma.

In [26]:
# Feature #2: Victorias y derrotas totales y anuales (YTD) - VERSIÓN CORREGIDA
from collections import defaultdict

# Diccionarios para tracking de estadísticas globales y anuales
player_stats = defaultdict(lambda: {'wins': 0, 'losses': 0})
player_stats_ytd = defaultdict(lambda: {'wins': 0, 'losses': 0, 'current_year': None})

# Listas para almacenar las estadísticas calculadas
player1_total_wins_list = []
player1_total_losses_list = []
player1_winrate_list = []
player1_ytd_wins_list = []
player1_ytd_losses_list = []
player1_ytd_winrate_list = []

player2_total_wins_list = []
player2_total_losses_list = []
player2_winrate_list = []
player2_ytd_wins_list = []
player2_ytd_losses_list = []
player2_ytd_winrate_list = []

print("Calculando estadísticas de victorias y derrotas...")
year_resets = 0  # Counter para debug

for idx, row in matches.iterrows():
    player1_name = row['player1_name']
    player2_name = row['player2_name']
    player1_won = row['player1_wins']
    year = row['year']  # ✅ USAR LA COLUMNA YEAR DIRECTAMENTE
    
    # --- PLAYER1 ---
    # Resetear YTD si cambió el año
    if player_stats_ytd[player1_name]['current_year'] is None:
        # Primera vez que vemos a este jugador
        player_stats_ytd[player1_name]['current_year'] = year
    elif player_stats_ytd[player1_name]['current_year'] != year:
        # Cambió el año, resetear
        player_stats_ytd[player1_name]['wins'] = 0
        player_stats_ytd[player1_name]['losses'] = 0
        player_stats_ytd[player1_name]['current_year'] = year
        year_resets += 1
    
    # Stats globales ANTES del partido (carrera completa)
    player1_total_wins = player_stats[player1_name]['wins']
    player1_total_losses = player_stats[player1_name]['losses']
    player1_total_matches = player1_total_wins + player1_total_losses
    player1_winrate = player1_total_wins / player1_total_matches if player1_total_matches > 0 else 0.0
    
    # Stats YTD ANTES del partido (solo del año actual)
    player1_ytd_wins = player_stats_ytd[player1_name]['wins']
    player1_ytd_losses = player_stats_ytd[player1_name]['losses']
    player1_ytd_matches = player1_ytd_wins + player1_ytd_losses
    player1_ytd_winrate = player1_ytd_wins / player1_ytd_matches if player1_ytd_matches > 0 else 0.0
    
    # Guardar valores de player1
    player1_total_wins_list.append(player1_total_wins)
    player1_total_losses_list.append(player1_total_losses)
    player1_winrate_list.append(player1_winrate)
    player1_ytd_wins_list.append(player1_ytd_wins)
    player1_ytd_losses_list.append(player1_ytd_losses)
    player1_ytd_winrate_list.append(player1_ytd_winrate)
    
    # --- PLAYER2 ---
    # Resetear YTD si cambió el año
    if player_stats_ytd[player2_name]['current_year'] is None:
        # Primera vez que vemos a este jugador
        player_stats_ytd[player2_name]['current_year'] = year
    elif player_stats_ytd[player2_name]['current_year'] != year:
        # Cambió el año, resetear
        player_stats_ytd[player2_name]['wins'] = 0
        player_stats_ytd[player2_name]['losses'] = 0
        player_stats_ytd[player2_name]['current_year'] = year
        year_resets += 1
    
    # Stats globales ANTES del partido (carrera completa)
    player2_total_wins = player_stats[player2_name]['wins']
    player2_total_losses = player_stats[player2_name]['losses']
    player2_total_matches = player2_total_wins + player2_total_losses
    player2_winrate = player2_total_wins / player2_total_matches if player2_total_matches > 0 else 0.0
    
    # Stats YTD ANTES del partido (solo del año actual)
    player2_ytd_wins = player_stats_ytd[player2_name]['wins']
    player2_ytd_losses = player_stats_ytd[player2_name]['losses']
    player2_ytd_matches = player2_ytd_wins + player2_ytd_losses
    player2_ytd_winrate = player2_ytd_wins / player2_ytd_matches if player2_ytd_matches > 0 else 0.0
    
    # Guardar valores de player2
    player2_total_wins_list.append(player2_total_wins)
    player2_total_losses_list.append(player2_total_losses)
    player2_winrate_list.append(player2_winrate)
    player2_ytd_wins_list.append(player2_ytd_wins)
    player2_ytd_losses_list.append(player2_ytd_losses)
    player2_ytd_winrate_list.append(player2_ytd_winrate)
    
    # --- Actualizar estadísticas DESPUÉS del partido ---
    if player1_won == 1:
        # Player1 gana
        player_stats[player1_name]['wins'] += 1
        player_stats_ytd[player1_name]['wins'] += 1
        # Player2 pierde
        player_stats[player2_name]['losses'] += 1
        player_stats_ytd[player2_name]['losses'] += 1
    else:
        # Player2 gana
        player_stats[player2_name]['wins'] += 1
        player_stats_ytd[player2_name]['wins'] += 1
        # Player1 pierde
        player_stats[player1_name]['losses'] += 1
        player_stats_ytd[player1_name]['losses'] += 1

# Añadir las columnas al DataFrame
matches['player1_total_wins'] = player1_total_wins_list
matches['player1_total_losses'] = player1_total_losses_list
matches['player1_winrate'] = player1_winrate_list
matches['player1_ytd_wins'] = player1_ytd_wins_list
matches['player1_ytd_losses'] = player1_ytd_losses_list
matches['player1_ytd_winrate'] = player1_ytd_winrate_list

matches['player2_total_wins'] = player2_total_wins_list
matches['player2_total_losses'] = player2_total_losses_list
matches['player2_winrate'] = player2_winrate_list
matches['player2_ytd_wins'] = player2_ytd_wins_list
matches['player2_ytd_losses'] = player2_ytd_losses_list
matches['player2_ytd_winrate'] = player2_ytd_winrate_list

print(f"✓ Columnas de victorias y derrotas añadidas al DataFrame.")
print(f"✓ Se detectaron {year_resets} reseteos de año YTD")

# Verificación de que YTD y Total son diferentes
print("\n🔍 Verificación de diferencias YTD vs Total:")
print(f"   Total wins - Media: {matches['player1_total_wins'].mean():.2f}, Max: {matches['player1_total_wins'].max()}")
print(f"   YTD wins   - Media: {matches['player1_ytd_wins'].mean():.2f}, Max: {matches['player1_ytd_wins'].max()}")
print(f"   Total winrate - Media: {matches['player1_winrate'].mean():.3f}")
print(f"   YTD winrate   - Media: {matches['player1_ytd_winrate'].mean():.3f}")

if matches['player1_ytd_wins'].max() < matches['player1_total_wins'].max():
    print("\n✅ YTD Max < Total Max → El YTD está funcionando correctamente!")
else:
    print("\n⚠️  YTD Max == Total Max → Revisar implementación...")

print("\nÚltimos 10 partidos con comparación YTD vs Total:")
display(matches[['player1_name', 'player2_name', 'year',
                 'player1_total_wins', 'player1_ytd_wins',
                 'player1_total_losses', 'player1_ytd_losses',
                 'player1_winrate', 'player1_ytd_winrate', 'player1_wins']].tail(10))

Calculando estadísticas de victorias y derrotas...
✓ Columnas de victorias y derrotas añadidas al DataFrame.
✓ Se detectaron 4362 reseteos de año YTD

🔍 Verificación de diferencias YTD vs Total:
   Total wins - Media: 90.94, Max: 812
   YTD wins   - Media: 11.44, Max: 80
   Total winrate - Media: 0.491
   YTD winrate   - Media: 0.479

✅ YTD Max < Total Max → El YTD está funcionando correctamente!

Últimos 10 partidos con comparación YTD vs Total:


,player1_name,player2_name,year,player1_total_wins,player1_ytd_wins,player1_total_losses,player1_ytd_losses,player1_winrate,player1_ytd_winrate,player1_wins
39531,Learner Tien,Arthur Fils,2024,4,4,5,3,0.444444,0.571429,1
39532,Jakub Mensik,Joao Fonseca,2024,28,25,19,18,0.595745,0.581395,0
39533,Joao Fonseca,Arthur Fils,2024,10,10,8,7,0.555556,0.588235,1
39534,Arthur Fils,Jakub Mensik,2024,62,37,46,28,0.574074,0.569231,1
39535,Luca Van Assche,Juncheng Shang,2024,20,8,38,18,0.344828,0.307692,1
39536,Nishesh Basavareddy,Luca Van Assche,2024,1,1,0,0,1.000000,1.000000,0
39537,Alex Michelsen,Learner Tien,2024,39,32,37,29,0.513158,0.524590,0
39538,Joao Fonseca,Learner Tien,2024,11,11,8,7,0.578947,0.611111,1
39539,Luca Van Assche,Alex Michelsen,2024,22,10,38,18,0.366667,0.357143,0
39540,Nishesh Basavareddy,Alex Michelsen,2024,1,1,1,1,0.500000,0.500000,0


### 3. Forma actual
Calculamos la proporción de victorias en los últimos 10 partidos de cada jugador. Esta feature refleja el estado de forma actual que es crucial porque muchos jugadores tienen rachas de confianza o bajones temporales que pueden afectar significativamente en sus resultados a corto plazo.

In [27]:
# Feature #3: Forma actual - Últimos 10 partidos
from collections import defaultdict, deque

# Diccionario para mantener los últimos resultados de cada jugador
# Usamos deque con maxlen=10 para mantener solo los últimos 10 partidos
player_recent_results = defaultdict(lambda: deque(maxlen=10))

# Listas para almacenar las estadísticas de forma
player1_last10_wins_list = []
player1_last10_losses_list = []
player1_last10_winrate_list = []

player2_last10_wins_list = []
player2_last10_losses_list = []
player2_last10_winrate_list = []

print("Calculando forma actual (últimos 10 partidos)...")
for idx, row in matches.iterrows():
    player1_name = row['player1_name']
    player2_name = row['player2_name']
    player1_won = row['player1_wins']
    
    # --- PLAYER1 ---
    # Obtener los últimos resultados ANTES del partido actual
    player1_recent = list(player_recent_results[player1_name])
    player1_last10_wins = sum(player1_recent)  # 1 = victoria, 0 = derrota
    player1_last10_losses = len(player1_recent) - player1_last10_wins
    player1_last10_winrate = player1_last10_wins / len(player1_recent) if len(player1_recent) > 0 else 0.0
    
    # Guardar valores
    player1_last10_wins_list.append(player1_last10_wins)
    player1_last10_losses_list.append(player1_last10_losses)
    player1_last10_winrate_list.append(player1_last10_winrate)
    
    # --- PLAYER2 ---
    # Obtener los últimos resultados ANTES del partido actual
    player2_recent = list(player_recent_results[player2_name])
    player2_last10_wins = sum(player2_recent)  # 1 = victoria, 0 = derrota
    player2_last10_losses = len(player2_recent) - player2_last10_wins
    player2_last10_winrate = player2_last10_wins / len(player2_recent) if len(player2_recent) > 0 else 0.0
    
    # Guardar valores
    player2_last10_wins_list.append(player2_last10_wins)
    player2_last10_losses_list.append(player2_last10_losses)
    player2_last10_winrate_list.append(player2_last10_winrate)
    
    # --- Actualizar resultados recientes DESPUÉS del partido ---
    if player1_won == 1:
        # Player1 gana (añadir 1)
        player_recent_results[player1_name].append(1)
        # Player2 pierde (añadir 0)
        player_recent_results[player2_name].append(0)
    else:
        # Player2 gana (añadir 1)
        player_recent_results[player2_name].append(1)
        # Player1 pierde (añadir 0)
        player_recent_results[player1_name].append(0)

# Añadir las columnas al DataFrame
matches['player1_last10_wins'] = player1_last10_wins_list
matches['player1_last10_losses'] = player1_last10_losses_list
matches['player1_last10_winrate'] = player1_last10_winrate_list

matches['player2_last10_wins'] = player2_last10_wins_list
matches['player2_last10_losses'] = player2_last10_losses_list
matches['player2_last10_winrate'] = player2_last10_winrate_list

print("✓ Columnas de forma actual añadidas al DataFrame.")
print("\nVerificación de las nuevas columnas:")
print(matches[['player1_name', 'player2_name', 'player1_last10_wins', 'player1_last10_losses', 
               'player1_last10_winrate', 'player2_last10_wins', 'player2_last10_losses', 'player2_last10_winrate', 'player1_wins']].tail(10))

Calculando forma actual (últimos 10 partidos)...
✓ Columnas de forma actual añadidas al DataFrame.

Verificación de las nuevas columnas:
              player1_name     player2_name  player1_last10_wins  \
39531         Learner Tien      Arthur Fils                    4   
39532         Jakub Mensik     Joao Fonseca                    6   
39533         Joao Fonseca      Arthur Fils                    7   
39534          Arthur Fils     Jakub Mensik                    5   
39535      Luca Van Assche   Juncheng Shang                    1   
39536  Nishesh Basavareddy  Luca Van Assche                    1   
39537       Alex Michelsen     Learner Tien                    6   
39538         Joao Fonseca     Learner Tien                    7   
39539      Luca Van Assche   Alex Michelsen                    3   
39540  Nishesh Basavareddy   Alex Michelsen                    1   

       player1_last10_losses  player1_last10_winrate  player2_last10_wins  \
39531                      5         

## Eliminar columnas con riesgo de data leakage

En este paso eliminamos todas las columnas originales `winner_*` y `loser_*`, que ya fueron transformadas a `player1_*` y `player2_*`, así como `score` y `minutes`. 
> score: contiene directamente el resultado del partido.

> time: refleja la duración del partido, información que solo se obtiene después del encuentro.

También se retiran todas las variables que contienen información posterior al resultado del partido, como estadísticas registradas después de conocer al ganador o perdedor, para evitar que el modelo “haga trampa” y sobrestime su rendimiento. Esta limpieza es fundamental para que el entrenamiento se base únicamente en datos disponibles antes del partido, garantizando así la validez del modelo y eliminando cualquier información directa del partido, como aces, dobles faltas o puntos ganados.

In [28]:
# Columnas con data leakage a eliminar
columnas_data_leakage = [
    'winner_aces',
    'winner_double_faults',
    'winner_serve_points_total',
    'winner_first_serves_in',
    'winner_first_serve_points_won',
    'winner_second_serve_points_won',
    'winner_service_games',
    'winner_break_points_saved',
    'winner_break_points_faced',
    'loser_aces',
    'loser_double_faults',
    'loser_serve_points_total',
    'loser_first_serves_in',
    'loser_first_serve_points_won',
    'loser_second_serve_points_won',
    'loser_service_games',
    'loser_break_points_saved',
    'loser_break_points_faced',
    'winner_name',
    'loser_name',
    'score',
    'minutes'
]

# Eliminar también las columnas player1_* y player2_* de estadísticas de partido
# (las que vienen del partido en sí, no las features engineered)
player_match_stats = [
    'player1_aces',
    'player1_double_faults',
    'player1_serve_points_total',
    'player1_first_serves_in',
    'player1_first_serve_points_won',
    'player1_second_serve_points_won',
    'player1_service_games',
    'player1_break_points_saved',
    'player1_break_points_faced',
    'player2_aces',
    'player2_double_faults',
    'player2_serve_points_total',
    'player2_first_serves_in',
    'player2_first_serve_points_won',
    'player2_second_serve_points_won',
    'player2_service_games',
    'player2_break_points_saved',
    'player2_break_points_faced',
]

# Combinar y eliminar solo las que existen
all_leakage_cols = columnas_data_leakage + player_match_stats
cols_to_drop = [col for col in all_leakage_cols if col in matches.columns]

print(f"Eliminando {len(cols_to_drop)} columnas con data leakage...")
matches.drop(columns=cols_to_drop, inplace=True)

print("✓ Columnas eliminadas")
print(f"\nColumnas restantes: {len(matches.columns)}")
print("\nPrimeras columnas del DataFrame limpio:")
display(matches.head())

Eliminando 4 columnas con data leakage...
✓ Columnas eliminadas

Columnas restantes: 79

Primeras columnas del DataFrame limpio:


,tourney_name,surface,draw_size,tourney_date,match_num,winner_entry,winner_hand,winner_ht,winner_ioc,winner_age,...,player2_winrate,player2_ytd_wins,player2_ytd_losses,player2_ytd_winrate,player1_last10_wins,player1_last10_losses,player1_last10_winrate,player2_last10_wins,player2_last10_losses,player2_last10_winrate
0,Brisbane,Hard,32,20110102,1,NaN,R,193.0,SWE,26.3,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
1,Brisbane,Hard,32,20110102,2,NaN,L,193.0,GER,30.5,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
2,Brisbane,Hard,32,20110102,3,Q,R,188.0,AUS,23.1,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
3,Brisbane,Hard,32,20110102,4,NaN,R,185.0,UZB,24.3,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
4,Brisbane,Hard,32,20110102,5,NaN,R,188.0,USA,29.0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0


Ahora revisaremos que el dataset esté completo y bien estructurado, comprobando el tamaño, que las categorías estén equilibradas y que las columnas sean coherentes y consistentes. Además, se eliminan posibles duplicados de las variables winner_* y loser_*, que ya fueron reemplazadas por las nuevas columnas. El objetivo es dejar el dataset completamente limpio y listo para la fase de modelado.

In [29]:
# Verificación final del dataset
print("="*80)
print("VERIFICACIÓN FINAL DEL DATASET")
print("="*80)

print(f"\n📊 Dimensiones: {matches.shape[0]:,} partidos × {matches.shape[1]} columnas")

print(f"\n🎯 Target balance:")
print(f"   player1_wins = 1: {matches['player1_wins'].sum():,} ({matches['player1_wins'].mean()*100:.2f}%)")
print(f"   player1_wins = 0: {(1-matches['player1_wins']).sum():,} ({(1-matches['player1_wins'].mean())*100:.2f}%)")

print(f"\n📝 Columnas del dataset:")
player1_cols = [col for col in matches.columns if col.startswith('player1_')]
player2_cols = [col for col in matches.columns if col.startswith('player2_')]
other_cols = [col for col in matches.columns if not col.startswith('player1_') and not col.startswith('player2_')]

print(f"\n   Player1 features ({len(player1_cols)}):")
for col in sorted(player1_cols):
    print(f"      - {col}")

print(f"\n   Player2 features ({len(player2_cols)}):")
for col in sorted(player2_cols):
    print(f"      - {col}")

print(f"\n   Otras columnas ({len(other_cols)}):")
for col in sorted(other_cols):
    print(f"      - {col}")

print("\n" + "="*80)
print("Dataset listo para el modelado ✅")
print("="*80)

VERIFICACIÓN FINAL DEL DATASET

📊 Dimensiones: 39,541 partidos × 79 columnas

🎯 Target balance:
   player1_wins = 1: 19,699 (49.82%)
   player1_wins = 0: 19,842 (50.18%)

📝 Columnas del dataset:

   Player1 features (19):
      - player1_age
      - player1_entry
      - player1_h2h_wins
      - player1_hand
      - player1_ht
      - player1_ioc
      - player1_last10_losses
      - player1_last10_winrate
      - player1_last10_wins
      - player1_name
      - player1_rank
      - player1_rank_points
      - player1_total_losses
      - player1_total_wins
      - player1_winrate
      - player1_wins
      - player1_ytd_losses
      - player1_ytd_winrate
      - player1_ytd_wins

   Player2 features (18):
      - player2_age
      - player2_entry
      - player2_h2h_wins
      - player2_hand
      - player2_ht
      - player2_ioc
      - player2_last10_losses
      - player2_last10_winrate
      - player2_last10_wins
      - player2_name
      - player2_rank
      - player2_rank_point

In [ ]:
# Limpiar columnas duplicadas winner_* y loser_* que quedaron del original
# (ya tenemos player1_* y player2_* que son las versiones con shuffle)
remaining_winner_loser_cols = [col for col in matches.columns 
                               if col.startswith('winner_') or col.startswith('loser_')]

if remaining_winner_loser_cols:
    print(f"Eliminando {len(remaining_winner_loser_cols)} columnas duplicadas winner_*/loser_*:")
    for col in remaining_winner_loser_cols:
        print(f"   - {col}")
    matches.drop(columns=remaining_winner_loser_cols, inplace=True)
    print(f"\n✓ Columnas eliminadas. Nuevas dimensiones: {matches.shape}")
else:
    print("✓ No hay columnas winner_*/loser_* adicionales para eliminar.")

Ahora dividimos entre entrenamiento y test, 70-30.

In [ ]:
matches_train, matches_test = train_test_split(matches, test_size=0.3, shuffle=False)

## Creación de la variable objetivo (Target)

El dataset ya contiene la columna `player1_wins` que indica si el jugador 1 ganó el partido (1) o perdió (0).

**Decisión de nomenclatura:**
- Mantenemos la estructura `player_1` vs `player_2` (estándar en deportes)
- Renombramos `player1_wins` → `winner` para mayor claridad
- **Interpretación:**
  - `winner = 1` → player_1 ganó el partido
  - `winner = 0` → player_2 ganó el partido

Esta variable será nuestro **target** para los modelos de clasificación.

In [30]:
# Verificar que la columna player1_wins existe
print("🔍 Verificando columna objetivo...")
if 'player1_wins' in matches.columns:
    print("✅ La columna 'player1_wins' existe en el dataset")
    
    # Renombrar para mayor claridad conceptual
    matches.rename(columns={'player1_wins': 'winner'}, inplace=True)
    matches_train.rename(columns={'player1_wins': 'winner'}, inplace=True)
    matches_test.rename(columns={'player1_wins': 'winner'}, inplace=True)
    
    print("✅ Columna renombrada: 'player1_wins' → 'winner'")
    print(f"\n📊 Balance del target:")
    print(f"   winner = 1 (player_1 ganó): {matches_train['winner'].sum():,} partidos ({matches_train['winner'].mean()*100:.2f}%)")
    print(f"   winner = 0 (player_2 ganó): {(~matches_train['winner'].astype(bool)).sum():,} partidos ({(1-matches_train['winner'].mean())*100:.2f}%)")
else:
    print("❌ ERROR: La columna 'player1_wins' no se encuentra en el dataset")
    print(f"Columnas disponibles: {matches.columns.tolist()}")

🔍 Verificando columna objetivo...
✅ La columna 'player1_wins' existe en el dataset


NameError: name 'matches_train' is not defined